In [1]:
import numpy as np
from typing import TypeAlias
import time

from hyperparameters import (
    _dt,
    _t_max,
    _tau_m,
    _V_reset,
    _V_thresh,
    _R,
    num_simulations
)

with open('weights.npy', 'rb') as f:
    W = np.load(f)

import jax.numpy as jnp
import jax

# Initial conditions
n_neurons = len(W)# Number of neurons in the network
_V = jnp.ones(n_neurons) * _V_reset  # Initial potentials

from tqdm.notebook import tqdm

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


# Type Definitions for Clarity

In [2]:
Tensor3D: TypeAlias = jnp.ndarray
Mat: TypeAlias = jnp.ndarray
Vec: TypeAlias = jnp.ndarray 

# Run the Simulations

In [8]:
@jax.jit
def run_step(
    v_prev,
    v_thresh,
    v_reset,

    W,
    tau_m,
    dt,
    membr_R,
):
    spiked = v_prev >= v_thresh
    V = jnp.where(spiked, v_reset, v_prev)

    # Update voltages
    I_syn = W.dot(spiked)  # Synaptic current from spikes
    dV = (dt / tau_m) * (-V + v_reset + membr_R * I_syn)
    V = V + dV

    # No self-inputs; neurons cannot spike themselves in this timestep
    V = jnp.where(spiked, v_reset, V)
    return V, spiked

jitted_res = jax.jit(run_step)

In [9]:

    
def run_simulation(
    W: Mat,
    V: Vec,

    # Neuron Parameters
    tau_m: float,
    v_reset: float,
    v_thresh: float,
    membr_R: float,

    # How long do we run for? 
    t_max: float,
    dt: float, 

):
    # Simulation

    spike_train = []
    for i, t in enumerate(np.arange(0, t_max, dt)):
        if i == 0:
            continue
        V, spike = jitted_res(V, v_thresh, v_reset, W, tau_m, dt, membr_R)
        spike_train.append(spike)

    return spike_train

time_arr = []
for i in range(num_simulations):
    start = time.time()
    spike_train = run_simulation(
        W,
        _V,
        _tau_m, _V_reset, _V_thresh, _R,
        _t_max, _dt
    )
    end = time.time()
    print(f"Iteration {i} took: {end - start} seconds")
    time_arr.append(end - start)

print(f"Average Time: {np.mean(time_arr)}")
print(f"S.Dev Time: {np.std(time_arr)}")

Iteration 0 took: 2.135218858718872 seconds
Iteration 1 took: 2.0883030891418457 seconds
Iteration 2 took: 2.1016340255737305 seconds


KeyboardInterrupt: 